In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper

### Exercise 1

Find the uncompressed SEC format for the Public Key where the Private Key secrets are:

* 5000
* \\(2018^{5}\\)
* 0xdeadbeef12345

In [14]:
# Exercise 1

from ecc import PrivateKey

# Define private key values
private_keys = [5000, 2018**5, 0xdeadbeef12345]

# Loop through each private key
for private_key in private_keys:
    # Create a PrivateKey object
    priv = PrivateKey(private_key)
    # Generate uncompressed SEC format and print the result
    print(priv.point.sec(compressed=False).hex())


04ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10
04027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9dff87647fd023c13b4a4994f17691895806e1b40b57f4fd22581a4f46851f3b06
04d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f842efa762fd59961d0e99803c61edba8b3e3f7dc3a341836f97733aebf987121


### Exercise 2

Find the Compressed SEC format for the Public Key where the Private Key secrets are:

* 5001
* \\(2019^{5}\\)
* 0xdeadbeef54321

In [18]:
# Exercise 2

from ecc import PrivateKey

# List of private keys
private_keys = [5001, 2019**5, 0xdeadbeef54321]

# Loop through each private key
for private_key in private_keys:
    # Create a PrivateKey object
    priv = PrivateKey(private_key)
    # Generate compressed SEC format and print the result
    print(priv.point.sec(compressed=True).hex())

0357a4f368868a8a6d572991e484e664810ff14c05c0fa023275251151fe0e53d1
02933ec2d2b111b92737ec12f1c5d20f3233a0ad21cd8b36d0bca7a0cfa5cb8701
0296be5b1292f6c856b3c5654e886fc13511462059089cdf9c479623bfcbe77690


### Exercise 3

Find the DER format for a signature whose `r` and `s` values are:

* r =

`0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6`

* s =

`0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec`

In [19]:
# Exercise 3

from ecc import Signature

# Given r and s values in hexadecimal
r_hex = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s_hex = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec

# Convert hexadecimal to integers
r = int(r_hex)
s = int(s_hex)

# Convert integers to bytes
r_bytes = r.to_bytes((r.bit_length() + 7) // 8, byteorder='big', signed=False)
s_bytes = s.to_bytes((s.bit_length() + 7) // 8, byteorder='big', signed=False)

# Calculate lengths
length_r = len(r_bytes)
length_s = len(s_bytes)
total_length = length_r + length_s + 4  # 4 bytes for headers and type indicators

# Generate DER format
der_format = bytes([0x30, total_length, 0x02, length_r]) + r_bytes + bytes([0x02, length_s]) + s_bytes

# Print DER format
print(der_format.hex())


3044022037206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c602208ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec


### Exercise 4

Convert the following hex to binary and then to Base58:

* `7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d`
* `eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c`
* `c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6`

In [21]:
# Exercise 4

from helper import encode_base58

# 7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d
# eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c
# c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6

# Hexadecimal strings
hex_strings = [ "7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d", "eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c"
,"c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6"]

for num in mylist:
    # Convert hexadecimal to binary
    binary_string = bytes.fromhex(num)
    # Convert binary to Base58
    base58_string = encode_base58(binary_string)
    # Print results
    print(base58_string)

9MA8fRQrT4u8Zj8ZRd6MAiiyaxb2Y1CMpvVkHQu5hVM6
4fE3H2E6XMp4SsxtwinF7w9a34ooUrwWe4WsW1458Pd
EQJsjkd6JaGwxrjEhfeqPenqHwrBmPQZjJGNSCHBkcF7


### Exercise 5

Find the address corresponding to Public Keys whose Private Key secrets are:

* 5002 (use uncompressed SEC, on testnet)
* \\(2020^{5}\\) (use compressed SEC, on testnet)
* 0x12345deadbeef (use compressed SEC on mainnet)

In [26]:
# Exercise 5

from ecc import PrivateKey
from helper import encode_base58_checksum, hash160

def generate_address(private_key, compressed, testnet):
    # Create a PrivateKey object
    priv = PrivateKey(private_key)
    # Get the public key in uncompressed or compressed format
    public_key = priv.point.sec(compressed=compressed)
    # Get the hash160 of the public key
    h160 = hash160(public_key)
    # Determine the network prefix (mainnet or testnet)
    prefix = b'\x6f' if testnet else b'\x00'
    # Concatenate the prefix and hash160, then encode in base58 with checksum
    address = encode_base58_checksum(prefix + h160)
    return address

# Private Key: 5002 (uncompressed, on testnet)
address_5002_testnet = generate_address(5002, compressed=False, testnet=True)
print("Private Key 5002 (uncompressed, on testnet):", address_5002_testnet)

# Private Key: 2020^5 (compressed, on testnet)
address_2020_5_testnet = generate_address(2020**5, compressed=True, testnet=True)
print("Private Key 2020^5 (compressed, on testnet):", address_2020_5_testnet)

# Private Key: 0x12345deadbeef (compressed, on mainnet)
address_12345deadbeef_mainnet = generate_address(0x12345deadbeef, compressed=True, testnet=False)
print("Private Key 0x12345deadbeef (compressed, on mainnet):", address_12345deadbeef_mainnet)



Private Key 5002 (uncompressed, on testnet): mmTPbXQFxboEtNRkwfh6K51jvdtHLxGeMA
Private Key 2020^5 (compressed, on testnet): mopVkxp8UhXqRYbCYJsbeE1h1fiF64jcoH
Private Key 0x12345deadbeef (compressed, on mainnet): 1F1Pn2y6pDb68E5nYJJeba4TLg2U7B6KF1


### Exercise 6

Find the WIF for Private Key whose secrets are:

* 5003 (compressed, testnet)
* \\(2021^{5}\\) (uncompressed, testnet)
* 0x54321deadbeef (compressed, mainnet)

In [27]:
# Exercise 6

from ecc import PrivateKey
from helper import encode_base58_checksum

def generate_wif(private_key, compressed, testnet):
    # Add prefix for testnet or mainnet
    prefix = b'\xef' if testnet else b'\x80'
    # Add suffix for compressed keys
    suffix = b'\x01' if compressed else b''
    # Concatenate the prefix, private key, and suffix
    wif_data = prefix + private_key.to_bytes(32, 'big') + suffix
    # Compute the checksum
    checksum = hash256(wif_data)[:4]
    # Concatenate the data and checksum, then encode in base58
    wif = encode_base58_checksum(wif_data + checksum)
    return wif

# Private Key: 5003 (compressed, testnet)
wif_5003_testnet = generate_wif(5003, compressed=True, testnet=True)
print("Private Key 5003 (compressed, testnet):", wif_5003_testnet)

# Private Key: 2021^5 (uncompressed, testnet)
wif_2021_5_testnet = generate_wif(2021**5, compressed=False, testnet=True)
print("Private Key 2021^5 (uncompressed, testnet):", wif_2021_5_testnet)

# Private Key: 0x54321deadbeef (compressed, mainnet)
wif_54321deadbeef_mainnet = generate_wif(0x54321deadbeef, compressed=True, testnet=False)
print("Private Key 0x54321deadbeef (compressed, mainnet):", wif_54321deadbeef_mainnet)


Private Key 5003 (compressed, testnet): 4zMHD81prP8vV95VyaL31Eja1WUZKXUuAqsn6fB1C5jCxD22PXctEbpCtD
Private Key 2021^5 (uncompressed, testnet): uR5aZQJBjdEUfzD18YxHNEiv8atGykUgPQHcu8ADZWiYSwBuRsGER25g
Private Key 0x54321deadbeef (compressed, mainnet): 38uMpGARR2BJy5p4dNFKYg9UsWNoBtkpbdrXEPTcMDkSMvFFg75eNhVGir


### Exercise 7

Write a function `little_endian_to_int` which takes Python bytes, interprets those bytes in Little-Endian and returns the number.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_little_endian_to_int`

In [28]:
# Exercise 7

reload(helper)
run(helper.HelperTest("test_little_endian_to_int"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 8

Write a function `int_to_little_endian` which does the reverse of the last exercise.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_int_to_little_endian`

In [29]:
# Exercise 8

reload(helper)
run(helper.HelperTest("test_int_to_little_endian"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 9

Create a testnet address for yourself using a long secret that only you know. This is important as there are bots on testnet trying to steal testnet coins. Make sure you write this secret down somewhere! You will be using the secret later to sign Transactions.

In [33]:
# Exercise 9

from ecc import PrivateKey
from helper import hash256, little_endian_to_int

# Select a passphrase 
passphrase = b'testing'

# Hash the passphrase using SHA-256 and convert to integer
secret = little_endian_to_int(hash256(passphrase))

# Create a private key using the derived secret
private_key = PrivateKey(secret)

# Get the public key from the private key
public_key = private_key.point

# Generate the testnet address from the public key
testnet_address = public_key.address(compressed=True, testnet=True)

# Print the testnet address
print(testnet_address)


n2PHmyMmFfJJb7WDQYe9VDnCK5iZQrvVga
